In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from glob import glob
import os
import cv2
import numpy as np
import cv2

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, concatenate, BatchNormalization, Dropout, Lambda , Activation, Conv2DTranspose
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [4]:
os.environ['PYTHONHASHSEED'] = '42'
np.random.seed(42)
tf.random.set_seed(42)

In [5]:
batch_size =6
learning_rate = 1e-4
epochs = 100
height = 240
width = 320


In [6]:
dataset_path = os.path.join('/content/drive/MyDrive/Colab Notebooks/cell_segmentation_and_classifier/Segmentation_data_and_labels')
files_dir = os.path.join('/content/drive/MyDrive/Colab Notebooks/cell_segmentation_and_classifier/Segmentation_data_and_labels/Segmentation_Data_files')

In [7]:
model_file = os.path.join(files_dir, 'unet-non-aug.h5')
log_file = os.path.join(files_dir, 'unet-non-aug.csv')

In [8]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)
create_dir(files_dir)

In [9]:

def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Dropout(0.3)(x)

    return x

In [10]:

def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPooling2D((2, 2))(x)
    return x, p

In [11]:
def decoder_block(input, skip_connection, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(input)
    x = concatenate([x, skip_connection], axis=3)
    x = conv_block(x, num_filters)
    return x


In [12]:
def build_unet(input_shape):
    inputs = tf.keras.layers.Input(input_shape)
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)
    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="U-Net")
    return model

In [13]:
def load_data(path):
    train_x = sorted(glob(os.path.join(path, "Segmentation_Data/*")))[:-5]
    train_y = sorted(glob(os.path.join(path, "Masks/*")))[:-5]

    val_x = sorted(glob(os.path.join(path, "Segmentation_Data/*")))[-5:]
    val_y = sorted(glob(os.path.join(path, "Masks/*")))[-5:]
    return (train_x, train_y), (val_x, val_y)


In [14]:

def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = x / 255.0
    return x.astype(np.float32)

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = x / 255.0
    x = np.expand_dims(x, axis=-1)
    return x.astype(np.float32)


In [15]:
def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y
    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([height, width, 3])
    y.set_shape([height, width, 1])

    return x, y

In [16]:
def tf_dataset(x, y, batch=6):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(tf_parse, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

(train_x, train_y), (val_x, val_y) = load_data(dataset_path)

In [17]:
(train_x, train_y), (val_x, val_y) = load_data(dataset_path)

In [18]:
print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Val: {len(val_x)} - {len(val_y)}")


Train: 18 - 18
Val: 5 - 5


In [22]:
def random_flip(x, y):
    flip_left_right = tf.random.uniform(shape=[], minval=0, maxval=1, dtype=tf.float32)
    flip_up_down = tf.random.uniform(shape=[], minval=0, maxval=1, dtype=tf.float32)

    x = tf.cond(flip_left_right > 0.5, lambda: tf.image.flip_left_right(x), lambda: x)
    y = tf.cond(flip_left_right > 0.5, lambda: tf.image.flip_left_right(y), lambda: y)

    x = tf.cond(flip_up_down > 0.5, lambda: tf.image.flip_up_down(x), lambda: x)
    y = tf.cond(flip_up_down > 0.5, lambda: tf.image.flip_up_down(y), lambda: y)

    return x, y

In [23]:
def tf_dataset(x, y, batch=6, augment=False):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(tf_parse, num_parallel_calls=tf.data.AUTOTUNE)
    if augment:
        dataset = dataset.map(lambda x, y: random_flip(x, y), num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

(train_x, train_y), (val_x, val_y) = load_data(dataset_path)

train_dataset = tf_dataset(train_x, train_y, batch=batch_size, augment=True)
val_dataset = tf_dataset(val_x, val_y, batch=batch_size, augment=True)

In [24]:
callbacks = [
    ModelCheckpoint(model_file, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10),
    CSVLogger(log_file),
]

In [25]:
model = build_unet(input_shape=(height, width, 3))

model.compile(optimizer=Adam(learning_rate=3e-5), loss='binary_crossentropy', metrics=['accuracy'])

model.fit(
    train_dataset,
    epochs=500,
    validation_data=val_dataset,
    callbacks=callbacks
)

Epoch 1/500
3/3 [==============================] - ETA: 0s - loss: 0.9749 - accuracy: 0.2682
Epoch 1: val_loss improved from inf to 0.70908, saving model to /content/drive/MyDrive/Colab Notebooks/cell_segmentation_and_classifier/Segmentation_data_and_labels/Segmentation_Data_files/unet-non-aug.h5
3/3 [==============================] - 23s 6s/step - loss: 0.9749 - accuracy: 0.2682 - val_loss: 0.7091 - val_accuracy: 0.0598 - lr: 3.0000e-05
Epoch 2/500
3/3 [==============================] - ETA: 0s - loss: 0.9248 - accuracy: 0.3114
Epoch 2: val_loss improved from 0.70908 to 0.70778, saving model to /content/drive/MyDrive/Colab Notebooks/cell_segmentation_and_classifier/Segmentation_data_and_labels/Segmentation_Data_files/unet-non-aug.h5
3/3 [==============================] - 5s 2s/step - loss: 0.9248 - accuracy: 0.3114 - val_loss: 0.7078 - val_accuracy: 0.0597 - lr: 3.0000e-05
Epoch 3/500
3/3 [==============================] - ETA: 0s - loss: 0.8819 - accuracy: 0.3627
Epoch 3: val_loss im